# Experiment Scene Setup

Load and preview pre-authored pick-and-place scenes stored as YAML directives.

## Notebook Goals

- Keep scenarios versioned as YAML files under `src/experiments_finalproject/generated_scenarios/`.
- Provide helpers to list available YAMLs and load a selected one via `LoadScenario`.
- Visualize each scene in MeshCat using the YAML description (no programmatic directive generation).

In [1]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, List

from pydrake.all import DiagramBuilder, StartMeshcat
from manipulation.station import LoadScenario, MakeHardwareStation

In [2]:
meshcat = StartMeshcat()
meshcat.Delete()  # start clean

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
PROJECT_ROOT = Path('/workspaces/RoboManip')
SCENARIO_DIR = PROJECT_ROOT / 'src/experiments_finalproject/generated_scenarios'
PUBLISH_EACH_SCENE = True
DEFAULT_SCENE = 'bin_to_bin_scene_0.yaml'

In [4]:
def ensure_scenario_dir() -> Path:
    if not SCENARIO_DIR.exists():
        raise FileNotFoundError(f"Scenario directory '{SCENARIO_DIR}' does not exist")
    return SCENARIO_DIR


def list_scenario_files(extension: str = '*.yaml') -> List[str]:
    ensure_scenario_dir()
    return sorted([p.name for p in SCENARIO_DIR.glob(extension)])


def load_scenario(filename: str) -> Dict[str, object]:
    ensure_scenario_dir()
    path = SCENARIO_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"Scenario file '{path}' not found")

    scenario = LoadScenario(filename=str(path))
    builder = DiagramBuilder()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    if PUBLISH_EACH_SCENE:
        diagram.ForcedPublish(context)

    return {'path': path, 'scenario': scenario, 'diagram': diagram, 'context': context, 'station': station}


In [5]:
available_scenarios = list_scenario_files()
available_scenarios

['bin_to_bin_scene_0.yaml',
 'stairs_scene_0.yaml',
 'table_to_table_scene_0.yaml']

In [6]:
if not available_scenarios:
    raise RuntimeError('No scenario YAML files found. Add files to generated_scenarios/.')

# Available scenarios:
# - bin_to_bin_scene_0.yaml: Pick and place from one bin to another
# - table_to_table_scene_0.yaml: Pick and place from one table to another
# - stairs_scene_0.yaml: Blocks organized as steps in a stair pattern

# Select a scenario by setting SELECTED_SCENE to one of the available_scenarios
SELECTED_SCENE = "stairs_scene_0.yaml"
print(f"Available scenarios: {available_scenarios}")
print(f"Selected scene: {SELECTED_SCENE}")
SELECTED_SCENE

Available scenarios: ['bin_to_bin_scene_0.yaml', 'stairs_scene_0.yaml', 'table_to_table_scene_0.yaml']
Selected scene: stairs_scene_0.yaml


'stairs_scene_0.yaml'

In [7]:
loaded_scene = load_scenario(SELECTED_SCENE)
loaded_scene['path']

PosixPath('/workspaces/RoboManip/src/experiments_finalproject/generated_scenarios/stairs_scene_0.yaml')

In [8]:
print(loaded_scene['path'].read_text())


directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.5]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.9]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [0.65]
        iiwa_joint_7: [1.7]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [-0.25, 0.0, 0.0]
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0.0, 0.0, 0.114]
        rotation: !Rpy { deg: [90.0, 0.0, 90.0] }
- add_model:
    name: table
    file: file:///workspaces/RoboManip/src/assets/models/manipulation_station/amazon_table.sdf
- add_weld:
    parent: world
    child: table::amazon_table
    X_PC:
        translation: [0.3257, 0.0, -0.0127]
# - add_model:
#     name: cupboard
#     file: packa

### Usage Notes

- Drop/edit YAML files in `src/experiments_finalproject/generated_scenarios/`, then rerun the listing cell to refresh options.
- Available scenarios:
  - **bin_to_bin_scene_0.yaml**: Pick and place from one bin to another
  - **table_to_table_scene_0.yaml**: Pick and place from one table to another
  - **stairs_scene_0.yaml**: Blocks organized as steps in a stair pattern
- Update `DEFAULT_SCENE` (or set `SELECTED_SCENE`) to load a different YAML file.
- `load_scenario` uses Drake's `LoadScenario` + `MakeHardwareStation`, so any resources referenced inside the YAML must exist in your environment.
- MeshCat output is overwritten each time you load a new scenario.
- Drake models (iiwa, wsg, cameras, cupboard) use `package://` paths. Local manipulation_station files (tables, bins, blocks) use `file://` paths.